## Deliverable 2. Create a Customer Travel Destinations Map.

In [39]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import API key
from config import places_api_key

# Configure gmaps API key
gmaps.configure(api_key=places_api_key)

In [40]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Zeya,RU,53.7500,127.2667,28.71,74,52,1.52,broken clouds
1,1,Ossora,RU,59.2353,163.0719,15.17,88,19,7.43,few clouds
2,2,Bystryy Istok,RU,52.3710,84.3873,61.27,30,100,4.81,overcast clouds
3,3,Nchelenge,ZM,-9.3451,28.7340,77.20,73,18,4.16,light rain
4,4,Aswan,EG,24.0934,32.9070,108.70,6,100,8.05,overcast clouds


In [41]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [42]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Nchelenge,ZM,-9.3451,28.7340,77.20,73,18,4.16,light rain
6,6,Kapaa,US,22.0752,-159.3190,70.12,97,17,12.33,moderate rain
8,8,Butaritari,KI,3.0707,172.7902,81.37,75,25,12.19,scattered clouds
12,12,Atuona,PF,-9.8000,-139.0333,79.20,76,59,17.65,broken clouds
13,13,Finschhafen,PG,-6.6024,147.8681,80.51,84,98,9.40,overcast clouds
15,15,Mumbwa,ZM,-14.9783,27.0619,78.48,57,50,5.68,scattered clouds
16,16,Beloha,MG,-25.1667,45.0500,78.33,71,53,14.58,broken clouds
18,18,Mahebourg,MU,-20.4081,57.7000,79.25,94,75,5.75,broken clouds
21,21,Rikitea,PF,-23.1203,-134.9692,77.90,68,6,11.83,clear sky
22,22,Pimenta Bueno,BR,-11.6725,-61.1936,79.14,79,86,4.74,overcast clouds


In [43]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                188
City                   188
Country                186
Lat                    188
Lng                    188
Max Temp               188
Humidity               188
Cloudiness             188
Wind Speed             188
Current Description    188
dtype: int64

In [44]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [45]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Nchelenge,ZM,77.20,light rain,-9.3451,28.7340,
6,Kapaa,US,70.12,moderate rain,22.0752,-159.3190,
8,Butaritari,KI,81.37,scattered clouds,3.0707,172.7902,
12,Atuona,PF,79.20,broken clouds,-9.8000,-139.0333,
13,Finschhafen,PG,80.51,overcast clouds,-6.6024,147.8681,
15,Mumbwa,ZM,78.48,scattered clouds,-14.9783,27.0619,
16,Beloha,MG,78.33,broken clouds,-25.1667,45.0500,
18,Mahebourg,MU,79.25,broken clouds,-20.4081,57.7000,
21,Rikitea,PF,77.90,clear sky,-23.1203,-134.9692,
22,Pimenta Bueno,BR,79.14,overcast clouds,-11.6725,-61.1936,


In [46]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": places_api_key,
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}" 
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [47]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!=""]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Nchelenge,ZM,77.20,light rain,-9.3451,28.7340,KD Travellers Home
6,Kapaa,US,70.12,moderate rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
8,Butaritari,KI,81.37,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
12,Atuona,PF,79.20,broken clouds,-9.8000,-139.0333,Villa Enata
13,Finschhafen,PG,80.51,overcast clouds,-6.6024,147.8681,Cossam Guest House


In [48]:
# 8a. Create the output File (CSV)
output_data_file = "../vacation_search/Vacation_Search.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [49]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [50]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))